In [1]:
import findspark
findspark.init()

In [2]:
# LIBRARY IMPORTS
import nltk
#nltk.download('wordnet')
import re
from nltk.corpus import stopwords
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
stop_words = set(stopwords.words('english'))
from pyspark.ml import Pipeline
from pyspark.sql import Row, Column, SparkSession
from textblob import TextBlob
import pyspark.sql.types as tp

In [3]:
sc = SparkContext(appName = "Text Cleaning")
strc = StreamingContext(sc, 3)
spark = SparkSession.builder.master("local[1]").appName("read tweet").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/18 02:27:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# LOAD DATA FROM TCP SOCKET
text_data = strc.socketTextStream('127.0.0.1', 5556)

In [5]:
#!pip install textblob
#!pip install elasticsearch
#!pip install kafka

In [5]:
# PRE-PROCESS DATA
def get_prediction(sentence): 
    sentence = sentence.lower()
    sentence = re.sub("\s+"," ", sentence)
    sentence = re.sub("\W"," ", sentence)
    sentence = re.sub(r"http\S+", "", sentence)
    sentence = ' '.join(word.lower() for word in sentence.split() if word not in stop_words and len(word) > 3)
    sentence = re.sub(r'\b\w{1,3}\b', '', sentence)
    res = TextBlob(sentence)
    senti = res.sentiment.polarity
    if senti > 0:
        senti = 'positive'
    elif senti < 0:
        senti = 'negative'
    elif senti == 0:
        senti = 'neutral'
    return str(senti)


In [6]:
# WRITE SENTIMENTS TO KAFKA TOPIC
def convt(rdd):
    
    my_sch = tp.StructType([tp.StructField(name= 'value',dataType= tp.StringType(),  nullable= True)])
    df = spark.createDataFrame([rdd], schema=my_sch)
    df.writeStream.format("console").start()
    
#     if rdd.isEmpty():
#         my_sch = tp.StructType([tp.StructField('value', tp.StringType(), True)])
#         rdd = sc.emptyRDD()
        
#         df = spark.createDataFrame(rdd, my_sch)  
#     else:
#         schema = tp.StructType([tp.StructField('value', tp.StringType(), True)])
#         rdd = sc.parallelize(rdd.collect())
#         rdd = rdd.map(lambda x:[x]) # transform the rdd
 
#         df = spark.createDataFrame(rdd, schema)

    
#     df.printSchema()
    
#     df.write.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("topic", "sentiments").save()

In [7]:
r = text_data.map(lambda x: get_prediction(x))
r.foreachRDD(convt)

r.pprint()

In [8]:
strc.start()
strc.awaitTermination()

22/04/18 02:29:43 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/18 02:29:43 WARN BlockManager: Block input-0-1650266982800 replicated to only 0 peer(s) instead of 1 peers
22/04/18 02:29:43 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/18 02:29:43 WARN BlockManager: Block input-0-1650266983000 replicated to only 0 peer(s) instead of 1 peers
22/04/18 02:29:43 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/18 02:29:43 WARN BlockManager: Block input-0-1650266983600 replicated to only 0 peer(s) instead of 1 peers


-------------------------------------------
Time: 2022-04-18 02:29:42
-------------------------------------------



22/04/18 02:29:44 ERROR JobScheduler: Error running job streaming job 1650266982000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/l5/x7kbn9s11053my1764cj0s700000gn/T/ipykernel_51235/2446481568.py", line 6, in convt
    df.writeStream.format("console").start()
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/sql/dataframe.py", line 264, in writeStream
    return DataStreamWriter(self)
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/sql/streaming.py", line 741, in __init__
    self._jwrite = df._jdf.writeStream()
  File "/Users/harishsrinivasan

Py4JJavaError: An error occurred while calling o23.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/l5/x7kbn9s11053my1764cj0s700000gn/T/ipykernel_51235/2446481568.py", line 6, in convt
    df.writeStream.format("console").start()
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/sql/dataframe.py", line 264, in writeStream
    return DataStreamWriter(self)
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/sql/streaming.py", line 741, in __init__
    self._jwrite = df._jdf.writeStream()
  File "/Users/harishsrinivasan/miniforge3/lib/py

22/04/18 02:29:44 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/18 02:29:44 WARN BlockManager: Block input-0-1650266984600 replicated to only 0 peer(s) instead of 1 peers
22/04/18 02:29:45 ERROR JobScheduler: Error running job streaming job 1650266985000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/l5/x7kbn9s11053my1764cj0s700000gn/T/ipykernel_51235/2446481568.py", line 6, in convt
    df.writeStream.format("console").start()
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/sql/dataframe.py", line 264, in writeStream
    return DataStre

...

-------------------------------------------
Time: 2022-04-18 03:20:03
-------------------------------------------



22/04/18 03:20:06 ERROR JobScheduler: Error running job streaming job 1650270006000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/l5/x7kbn9s11053my1764cj0s700000gn/T/ipykernel_51235/2446481568.py", line 6, in convt
    df.writeStream.format("console").start()
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/sql/dataframe.py", line 264, in writeStream
    return DataStreamWriter(self)
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/sql/streaming.py", line 741, in __init__
    self._jwrite = df._jdf.writeStream()
  File "/Users/harishsrinivasan

-------------------------------------------
Time: 2022-04-18 03:20:06
-------------------------------------------



22/04/18 03:20:09 ERROR JobScheduler: Error running job streaming job 1650270009000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/l5/x7kbn9s11053my1764cj0s700000gn/T/ipykernel_51235/2446481568.py", line 6, in convt
    df.writeStream.format("console").start()
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/sql/dataframe.py", line 264, in writeStream
    return DataStreamWriter(self)
  File "/Users/harishsrinivasan/miniforge3/lib/python3.9/site-packages/pyspark/sql/streaming.py", line 741, in __init__
    self._jwrite = df._jdf.writeStream()
  File "/Users/harishsrinivasan

-------------------------------------------
Time: 2022-04-18 03:20:09
-------------------------------------------

